## Continuous Multi-Fidelity BO in BoTorch with Knowledge Gradient

In this tutorial, we show how to perform continuous multi-fidelity Bayesian optimization (BO) in BoTorch using the multi-fidelity Knowledge Gradient (qMFKG) acquisition function [1, 2].

[1] [J. Wu, P.I. Frazier. Continuous-Fidelity Bayesian Optimization with Knowledge Gradient. NIPS Workshop on Bayesian Optimization, 2017.](https://bayesopt.github.io/papers/2017/20.pdf)

[2] [J. Wu, S. Toscano-Palmerin, P.I. Frazier, A.G. Wilson. Practical Multi-fidelity Bayesian Optimization for Hyperparameter Tuning. Conference on Uncertainty in Artificial Intelligence (UAI), 2019](https://arxiv.org/pdf/1903.04703.pdf)

### Set dtype and device

In [2]:
import os
import torch


tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST")

### Problem setup

We'll consider the Augmented Hartmann multi-fidelity synthetic test problem. This function is a version of the Hartmann6 test function with an additional dimension representing the fidelity parameter; details are in [2]. The function takes the form $f(x,s)$ where $x \in [0,1]^6$ and $s \in [0,1]$. The target fidelity is 1.0, which means that our goal is to solve $\max_x f(x,1.0)$ by making use of cheaper evaluations $f(x,s)$ for $s < 1.0$. In this example, we'll assume that the cost function takes the form $5.0 + s$, illustrating a situation where the fixed cost is $5.0$.

In [3]:
from botorch.test_functions.multi_fidelity import AugmentedHartmann


problem = AugmentedHartmann(negate=True).to(**tkwargs)

c:\cond\envs\MFBO\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Model initialization

We use a `SingleTaskMultiFidelityGP` as the surrogate model, which uses a kernel from [2] that is well-suited for multi-fidelity applications.

In [4]:
from botorch.models.gp_regression_fidelity import SingleTaskMultiFidelityGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.utils.transforms import unnormalize
from botorch.utils.sampling import draw_sobol_samples


def generate_initial_data(n=16):
    # generate training data
    train_x = torch.rand(n, 7, **tkwargs)
    train_obj = problem(train_x).unsqueeze(-1)  # add output dimension
    return train_x, train_obj


def initialize_model(train_x, train_obj):
    # define a surrogate model suited for a "training data"-like fidelity parameter
    # in dimension 6, as in [2]
    model = SingleTaskMultiFidelityGP(
        train_x, train_obj, outcome_transform=Standardize(m=1), data_fidelities=[6]
    )
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    return mll, model

#### Define a helper function to construct the MFKG acquisition function
The helper function illustrates how one can initialize a $q$MFKG acquisition function. In this example, we assume that the affine cost is known. We then use the notion of a `CostAwareUtility` in BoTorch to scalarize the competing objectives of information gain and cost. The MFKG acquisition function optimizes the ratio of information gain to cost, which is captured by the `InverseCostWeightedUtility`.

In order for MFKG to evaluate the information gain, it uses the model to predict the function value at the highest fidelity after conditioning on the observation. This is handled by the `project` argument, which specifies how to transform a tensor `X` to its target fidelity. We use a default helper function called `project_to_target_fidelity` to achieve this.

An important point to keep in mind: in the case of standard KG, one can ignore the current value and simply optimize the expected maximum posterior mean of the next stage. However, for MFKG, since the goal is optimize information *gain* per cost, it is important to first compute the current value (i.e., maximum of the posterior mean at the target fidelity). To accomplish this, we use a `FixedFeatureAcquisitionFunction` on top of a `PosteriorMean`.

In [5]:
from botorch import fit_gpytorch_mll
from botorch.models.cost import AffineFidelityCostModel
from botorch.acquisition.cost_aware import InverseCostWeightedUtility
from botorch.acquisition import PosteriorMean
from botorch.acquisition.knowledge_gradient import qMultiFidelityKnowledgeGradient
from botorch.acquisition.fixed_feature import FixedFeatureAcquisitionFunction
from botorch.optim.optimize import optimize_acqf
from botorch.acquisition.utils import project_to_target_fidelity


bounds = torch.tensor([[0.0] * problem.dim, [1.0] * problem.dim], **tkwargs)
target_fidelities = {6: 1.0}

cost_model = AffineFidelityCostModel(fidelity_weights={6: 1.0}, fixed_cost=5.0)
cost_aware_utility = InverseCostWeightedUtility(cost_model=cost_model)


def project(X):
    return project_to_target_fidelity(X=X, target_fidelities=target_fidelities)


def get_mfkg(model):

    curr_val_acqf = FixedFeatureAcquisitionFunction(
        acq_function=PosteriorMean(model),
        d=7,
        columns=[6],
        values=[1],
    )

    _, current_value = optimize_acqf(
        acq_function=curr_val_acqf,
        bounds=bounds[:, :-1],
        q=1,
        num_restarts=10 if not SMOKE_TEST else 2,
        raw_samples=1024 if not SMOKE_TEST else 4,
        options={"batch_limit": 10, "maxiter": 200},
    )

    return qMultiFidelityKnowledgeGradient(
        model=model,
        num_fantasies=128 if not SMOKE_TEST else 2,
        current_value=current_value,
        cost_aware_utility=cost_aware_utility,
        project=project,
    )

#### Define a helper function that performs the essential BO step
This helper function optimizes the acquisition function and returns the batch $\{x_1, x_2, \ldots x_q\}$ along with the observed function values. 

In [6]:
from botorch.optim.initializers import gen_one_shot_kg_initial_conditions

torch.set_printoptions(precision=3, sci_mode=False)

NUM_RESTARTS = 10 if not SMOKE_TEST else 2
RAW_SAMPLES = 512 if not SMOKE_TEST else 4


def optimize_mfkg_and_get_observation(mfkg_acqf):
    """Optimizes MFKG and returns a new candidate, observation, and cost."""

    X_init = gen_one_shot_kg_initial_conditions(
        acq_function=mfkg_acqf,
        bounds=bounds,
        q=4,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
    )
    candidates, _ = optimize_acqf(
        acq_function=mfkg_acqf,
        bounds=bounds,
        q=4,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        batch_initial_conditions=X_init,
        options={"batch_limit": 5, "maxiter": 200},
    )
    # observe new values
    cost = cost_model(candidates).sum()
    new_x = candidates.detach()
    new_obj = problem(new_x).unsqueeze(-1)
    print(f"candidates:\n{new_x}\n")
    print(f"observations:\n{new_obj}\n\n")
    return new_x, new_obj, cost

### Perform a few steps of multi-fidelity BO
First, let's generate some initial random data and fit a surrogate model.

In [7]:
train_x, train_obj = generate_initial_data(n=16)

We can now use the helper functions above to run a few iterations of BO.

In [15]:
import timeit

cumulative_cost = 0.0
N_ITER = 6 if not SMOKE_TEST else 2


for _ in range(N_ITER):
    start_time = timeit.default_timer()
    mll, model = initialize_model(train_x, train_obj)
    print(f"Time to initialize model: {timeit.default_timer()-start_time:.2f}")
    fit_gpytorch_mll(mll)
    print(f"Time to fit model: {timeit.default_timer()-start_time:.2f}")
    mfkg_acqf = get_mfkg(model)
    print(f"Time to get acqf: {timeit.default_timer()-start_time:.2f}")
    new_x, new_obj, cost = optimize_mfkg_and_get_observation(mfkg_acqf)
    print(f"Time to optimize acqf: {timeit.default_timer()-start_time:.2f}")
    train_x = torch.cat([train_x, new_x])
    train_obj = torch.cat([train_obj, new_obj])
    cumulative_cost += cost

Time to initialize model: 0.01
Time to fit model: 2.40
Time to get acqf: 3.84
candidates:
tensor([[0.346, 0.127, 0.525, 0.325, 0.335, 0.689, 0.961],
        [0.255, 0.547, 0.270, 0.115, 0.402, 0.155, 0.345],
        [0.371, 0.120, 0.377, 0.326, 0.297, 0.712, 0.969],
        [0.285, 0.083, 0.422, 0.248, 0.306, 0.664, 0.964]],
       dtype=torch.float64)

observations:
tensor([[2.940],
        [0.240],
        [2.865],
        [3.151]], dtype=torch.float64)


Time to optimize acqf: 52.76
Time to initialize model: 0.01
Time to fit model: 1.78
Time to get acqf: 2.67
candidates:
tensor([[0.265, 0.077, 0.543, 0.233, 0.271, 0.672, 0.961],
        [0.200, 0.054, 0.475, 0.287, 0.298, 0.678, 0.945],
        [0.077, 0.151, 0.372, 0.213, 0.289, 0.732, 0.591],
        [0.445, 0.288, 0.831, 0.594, 0.114, 0.197, 0.515]],
       dtype=torch.float64)

observations:
tensor([[3.030],
        [3.195],
        [2.764],
        [0.193]], dtype=torch.float64)


Time to optimize acqf: 52.49
Time to initialize

### Make a final recommendation
In multi-fidelity BO, there are usually fewer observations of the function at the target fidelity, so it is important to use a recommendation function that uses the correct fidelity. Here, we maximize the posterior mean with the fidelity dimension fixed to the target fidelity of 1.0.

In [9]:

def get_recommendation(model):

    rec_acqf = FixedFeatureAcquisitionFunction(
        acq_function=PosteriorMean(model),
        d=7,
        columns=[6],
        values=[1],
    )

    final_rec, _ = optimize_acqf(
        acq_function=rec_acqf,
        bounds=bounds[:, :-1],
        q=1,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit": 5, "maxiter": 200},
    )

    final_rec = rec_acqf._construct_X_full(final_rec)

    objective_value = problem(final_rec)
    print(f"recommended point:\n{final_rec}\n\nobjective value:\n{objective_value}")
    return final_rec

In [10]:
final_rec = get_recommendation(model)
print(f"\ntotal cost: {cumulative_cost}\n")

recommended point:
tensor([[0.304, 0.153, 0.522, 0.279, 0.307, 0.692, 1.000]],
       dtype=torch.float64)

objective value:
tensor([3.148], dtype=torch.float64)

total cost: 122.32543738192216



### Comparison to standard EI (always use target fidelity)
Let's now repeat the same steps using a standard EI acquisition function (note that this is not a rigorous comparison as we are only looking at one trial in order to keep computational requirements low).

In [16]:
from botorch.acquisition import qExpectedImprovement

def get_ei(model, best_f):

    return FixedFeatureAcquisitionFunction(
        acq_function=qExpectedImprovement(model=model, best_f=best_f),
        d=7,
        columns=[6],
        values=[1],
    )


def optimize_ei_and_get_observation(ei_acqf):
    """Optimizes EI and returns a new candidate, observation, and cost."""

    candidates, _ = optimize_acqf(
        acq_function=ei_acqf,
        bounds=bounds[:, :-1],
        q=4,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit": 5, "maxiter": 200},
    )

    # add the fidelity parameter
    candidates = ei_acqf._construct_X_full(candidates)

    # observe new values
    cost = cost_model(candidates).sum()
    new_x = candidates.detach()
    new_obj = problem(new_x).unsqueeze(-1)
    #print(f"candidates:\n{new_x}\n")
    #print(f"observations:\n{new_obj}\n\n")
    return new_x, new_obj, cost

In [17]:
import timeit

cumulative_cost = 0.0

train_x, train_obj = generate_initial_data(n=16)

for _ in range(N_ITER):
    start = timeit.timeit()
    mll, model = initialize_model(train_x, train_obj)
    print(f"model initialization: {timeit.timeit() - start}")
    fit_gpytorch_mll(mll)
    print(f"model fitting: {timeit.timeit() - start}")
    ei_acqf = get_ei(model, best_f=train_obj.max())
    print(f"acqf initialization: {timeit.timeit() - start}")
    new_x, new_obj, cost = optimize_ei_and_get_observation(ei_acqf)
    print(f"acqf optimization: {timeit.timeit() - start}")
    train_x = torch.cat([train_x, new_x])
    train_obj = torch.cat([train_obj, new_obj])
    cumulative_cost += cost

model initialization: 0.002168899984098971
model fitting: -0.00404780000098981
acqf initialization: 0.004791799990925938
acqf optimization: -0.003980100009357557
model initialization: 0.006471200016676448
model fitting: 0.005179600018891506
acqf initialization: 0.017366300016874447
acqf optimization: 0.013926500017987564
model initialization: -0.0004281000146875158
model fitting: -0.006543200011947192
acqf initialization: -0.004853100006585009
acqf optimization: -0.01343140001699794
model initialization: 0.0051488000026438385
model fitting: 0.00963479999336414
acqf initialization: 0.0059591999888652936
acqf optimization: 0.008068099996307865
model initialization: 0.005562499994994141
model fitting: -0.01289080000424292
acqf initialization: -0.013044899998931214
acqf optimization: -0.009928700004820712
model initialization: -0.01314490000368096
model fitting: -0.001228799985256046
acqf initialization: -0.015086599989444949
acqf optimization: 0.007683500007260591


In [12]:
final_rec = get_recommendation(model)
print(f"\ntotal cost: {cumulative_cost}\n")

recommended point:
tensor([[0.288, 0.175, 0.520, 0.283, 0.351, 0.735, 1.000]],
       dtype=torch.float64)

objective value:
tensor([2.990], dtype=torch.float64)

total cost: 144.0

